In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization)
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


df = pd.read_csv("age_gender.csv")


def process_pixels(pixel_str):
    pixels = np.array(pixel_str.split(), dtype="uint8")
    return pixels.reshape(48, 48, 1)

df["pixels"] = df["pixels"].apply(process_pixels)


X = np.stack(df["pixels"].values) / 255.0  
y_age = df["age"].values  
y_gender = df["gender"].values 
y_ethnicity = to_categorical(df["ethnicity"].values) 


X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test, y_ethnicity_train, y_ethnicity_test = train_test_split(
    X, y_age, y_gender, y_ethnicity, test_size=0.2, random_state=42
)


input_layer = Input(shape=(48, 48, 1))

x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)


age_output = Dense(1, activation="linear", name="age_output")(x)  
gender_output = Dense(1, activation="sigmoid", name="gender_output")(x)  
ethnicity_output = Dense(y_ethnicity.shape[1], activation="softmax", name="ethnicity_output")(x)  


model = Model(inputs=input_layer, outputs=[age_output, gender_output, ethnicity_output])


model.compile(
    optimizer="adam",
    loss={
        "age_output": "mse",  
        "gender_output": "binary_crossentropy",  
        "ethnicity_output": "categorical_crossentropy", 
    },
    metrics={
        "age_output": "mae",  
        "gender_output": "accuracy",
        "ethnicity_output": "accuracy",
    }
)

# Train model
model.fit(
    X_train,
    {"age_output": y_age_train, "gender_output": y_gender_train, "ethnicity_output": y_ethnicity_train},
    validation_data=(X_test, {"age_output": y_age_test, "gender_output": y_gender_test, "ethnicity_output": y_ethnicity_test}),
    epochs=20,
    batch_size=32
)


model.save("multi_output_cnn.keras")


Epoch 1/20
593/593 ━━━━━━━━━━━━━━━━━━━━ 47s 68ms/step - age_output_loss: 290.2667 - age_output_mae: 12.7258 - ethnicity_output_accuracy: 0.3216 - ethnicity_output_loss: 5.3128 - gender_output_accuracy: 0.5686 - gender_output_loss: 2.1440 - loss: 297.7237 - val_age_output_loss: 269.1617 - val_age_output_mae: 12.3734 - val_ethnicity_output_accuracy: 0.4927 - val_ethnicity_output_loss: 1.3271 - val_gender_output_accuracy: 0.5554 - val_gender_output_loss: 0.6778 - val_loss: 271.0390
Epoch 2/20
593/593 ━━━━━━━━━━━━━━━━━━━━ 39s 66ms/step - age_output_loss: 149.3585 - age_output_mae: 9.0246 - ethnicity_output_accuracy: 0.4068 - ethnicity_output_loss: 1.5238 - gender_output_accuracy: 0.6253 - gender_output_loss: 0.6707 - loss: 151.5531 - val_age_output_loss: 338.7094 - val_age_output_mae: 15.4376 - val_ethnicity_output_accuracy: 0.4881 - val_ethnicity_output_loss: 1.3325 - val_gender_output_accuracy: 0.6100 - val_gender_output_loss: 0.6643 - val_loss: 341.0919
Epoch 3/20
593/593 ━━━━━━━━━━━━━━

In [3]:
model.save("multi_output_cnn.keras")


In [5]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("multi_output_cnn.keras")


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
import tensorflow as tf

# Load model (change filename if using .keras)
MODEL_PATH = "multi_output_cnn.keras"
model = tf.keras.models.load_model(MODEL_PATH)

# Print model summary
model.summary()


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 48, 48,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 48, 48,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 24, 24,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 24, 24,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 24, 24,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 12,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 12,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 12,    │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 6, 6, 128) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 4608)      │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    589,952 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ age_output (Dense)  │ (None, 1)         │        129 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gender_output       │ (None, 1)         │        129 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ethnicity_output    │ (None, 5)         │        645 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,368,400 (5.22 MB)

 Trainable params: 683,975 (2.61 MB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 683,977 (2.61 MB)

In [5]:
import numpy as np

# Create a dummy grayscale image (48x48)
dummy_image = np.random.rand(1, 48, 48, 1)  # Batch size 1

# Make prediction
age_pred, gender_pred, ethnicity_pred = model.predict(dummy_image)

# Process outputs
age = round(age_pred[0][0])  # Convert to integer
gender = "Female" if gender_pred[0][0] > 0.5 else "Male"
ethnicity = np.argmax(ethnicity_pred[0])  # Get class index

print(f"Predicted Age: {age}")
print(f"Predicted Gender: {gender}")
print(f"Predicted Ethnicity Class: {ethnicity}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
Predicted Age: 48
Predicted Gender: Male
Predicted Ethnicity Class: 1


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_image(image_path):
    img = load_img(image_path, color_mode="grayscale", target_size=(48, 48))
    img = img_to_array(img) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Path to your test image
image_path = "your_image.jpg"
input_image = preprocess_image(image_path)

# Make prediction
age_pred, gender_pred, ethnicity_pred = model.predict(input_image)

# Process results
age = round(age_pred[0][0])
gender = "Female" if gender_pred[0][0] > 0.5 else "Male"
ethnicity = np.argmax(ethnicity_pred[0])

print(f"Predicted Age: {age}")
print(f"Predicted Gender: {gender}")
print(f"Predicted Ethnicity Class: {ethnicity}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
Predicted Age: 51
Predicted Gender: Male
Predicted Ethnicity Class: 1


In [17]:
import numpy as np
import cv2

# Load the trained model
model = tf.keras.models.load_model("multi_output_cnn.keras")

# Function to preprocess input image
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img = cv2.resize(img, (48, 48))  # Resize to match model input
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=[0, -1])  # Expand dimensions for model input
    return img

# Predict on a test image
img_path = "mohith.jpg"  # Replace with actual image path
img = preprocess_image(img_path)
age_pred, gender_pred, ethnicity_pred = model.predict(img)

# Convert predictions to human-readable format
predicted_age = int(age_pred[0][0])
predicted_gender = "Male" if gender_pred[0][0] > 0.5 else "Female"
predicted_ethnicity = np.argmax(ethnicity_pred[0])  # Get highest probability class

print(f"Predicted Age: {predicted_age}")
print(f"Predicted Gender: {predicted_gender}")
print(f"Predicted Ethnicity Class: {predicted_ethnicity}")


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
Predicted Age: 50
Predicted Gender: Female
Predicted Ethnicity Class: 1


In [15]:
import numpy as np
import cv2
import tensorflow as tf

# Load the fine-tuned model
model = tf.keras.models.load_model("multi_output_cnn.keras")

# Define ethnicity class labels
ethnicity_classes = {0: "White", 1: "Black", 2: "Asian", 3: "Indian", 4: "Others"}

# Function to preprocess input image
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img = cv2.resize(img, (48, 48))  # Resize to match model input
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=[0, -1])  # Expand dimensions for model input
    return img

# Predict on a new image
def predict_image(img_path):
    img = preprocess_image(img_path)
    age_pred, gender_pred, ethnicity_pred = model.predict(img)

    # Convert predictions
    predicted_age = int(age_pred[0][0])  # Age output
    predicted_gender = "Male" if gender_pred[0][0] > 0.5 else "Female"  # Gender classification
    predicted_ethnicity = ethnicity_classes[np.argmax(ethnicity_pred[0])]  # Ethnicity classification

    print(f"Predicted Age: {predicted_age}")
    print(f"Predicted Gender: {predicted_gender}")
    print(f"Predicted Ethnicity: {predicted_ethnicity}")

# Test the script with an image
image_path = "mohith.jpg"  # Replace with actual image path
predict_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
Predicted Age: 50
Predicted Gender: Female
Predicted Ethnicity: Black


In [1]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("multi_output_cnn.keras")

# Unfreeze some layers for fine-tuning
for layer in model.layers[-10:]:  # Unfreezing last 10 layers
    if hasattr(layer, "trainable"):
        layer.trainable = True

# Recompile with a lower learning rate for fine-tuning
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss={
        "age_output": "mse",
        "gender_output": "binary_crossentropy",
        "ethnicity_output": "categorical_crossentropy",
    },
    metrics={
        "age_output": "mae",
        "gender_output": "accuracy",
        "ethnicity_output": "accuracy",
    }
)


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.2
)

datagen.fit(X_train)


NameError: name 'X_train' is not defined

In [5]:
import numpy as np
import pandas as pd
df=pd.read_csv("age_gender.csv")
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.2
)

datagen.fit(X_train)


NameError: name 'X_train' is not defined

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Load dataset again
df = pd.read_csv("age_gender.csv")

# Process pixel data
def process_pixels(pixel_str):
    return np.array(pixel_str.split(), dtype="uint8").reshape(48, 48, 1)

df["pixels"] = df["pixels"].apply(process_pixels)

# Normalize images
X = np.stack(df["pixels"].values) / 255.0  

# Extract labels
y_age = df["age"].values / df["age"].max()  # Normalize age
y_gender = df["gender"].values
y_ethnicity = to_categorical(df["ethnicity"].values)

# Train-test split
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test, y_ethnicity_train, y_ethnicity_test = train_test_split(
    X, y_age, y_gender, y_ethnicity, test_size=0.2, random_state=42
)

# Compute class weights for gender (handle imbalanced data)
class_weights = compute_class_weight("balanced", classes=np.unique(y_gender), y=y_gender)
gender_class_weights = dict(enumerate(class_weights))

# Load the saved model
model = load_model("multi_output_cnn.keras")

# Unfreeze last 10 layers for fine-tuning
for layer in model.layers[-10:]:
    if hasattr(layer, "trainable"):
        layer.trainable = True

# Recompile with a lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Small learning rate for fine-tuning
    loss={
        "age_output": "mse",
        "gender_output": "binary_crossentropy",
        "ethnicity_output": "categorical_crossentropy",
    },
    metrics={
        "age_output": "mae",
        "gender_output": "accuracy",
        "ethnicity_output": "accuracy",
    }
)

# Data Augmentation for Fine-Tuning
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.2
)
datagen.fit(X_train)

# Callbacks for fine-tuning
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)

# Continue training (fine-tuning)
model.fit(
    datagen.flow(X_train, {"age_output": y_age_train, "gender_output": y_gender_train, "ethnicity_output": y_ethnicity_train}, batch_size=64),
    validation_data=(X_test, {"age_output": y_age_test, "gender_output": y_gender_test, "ethnicity_output": y_ethnicity_test}),
    epochs=30,  # Increase epochs for fine-tuning
    steps_per_epoch=len(X_train) // 64,  # Ensures full dataset coverage
    class_weight={"gender_output": gender_class_weights},
    callbacks=[lr_scheduler, early_stopping]
)

# Save the fine-tuned model
model.save("multi_output_cnn_finetuned.keras")


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (18964, 48, 48, 1), y.shape = ()

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Load dataset
df = pd.read_csv("age_gender.csv")

# Process pixel data
def process_pixels(pixel_str):
    return np.array(pixel_str.split(), dtype="uint8").reshape(48, 48, 1)

df["pixels"] = df["pixels"].apply(process_pixels)

# Normalize images
X = np.stack(df["pixels"].values) / 255.0  

# Ensure labels are correctly shaped
y_age = df["age"].values.astype(np.float32) / df["age"].max()  # Normalize age
y_gender = df["gender"].values.astype(np.float32)  # Gender (0 or 1)
y_ethnicity = to_categorical(df["ethnicity"].values)  # One-hot encode ethnicity

# Print shapes to verify
print(f"X shape: {X.shape}")
print(f"y_age shape: {y_age.shape}")
print(f"y_gender shape: {y_gender.shape}")
print(f"y_ethnicity shape: {y_ethnicity.shape}")

# Train-test split with stratification on gender
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test, y_ethnicity_train, y_ethnicity_test = train_test_split(
    X, y_age, y_gender, y_ethnicity, test_size=0.2, random_state=42, stratify=y_gender
)

# Compute class weights for gender (handle imbalanced data)
class_weights = compute_class_weight("balanced", classes=np.unique(y_gender_train), y=y_gender_train)
gender_class_weights = dict(enumerate(class_weights))

# Load the saved model
model = load_model("multi_output_cnn.keras")

# Unfreeze last 10 layers for fine-tuning
for layer in model.layers[-10:]:
    if hasattr(layer, "trainable"):
        layer.trainable = True

# Recompile with a lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Small learning rate for fine-tuning
    loss={
        "age_output": "mse",
        "gender_output": "binary_crossentropy",
        "ethnicity_output": "categorical_crossentropy",
    },
    metrics={
        "age_output": "mae",
        "gender_output": "accuracy",
        "ethnicity_output": "accuracy",
    }
)

# Data Augmentation for Fine-Tuning
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.2
)
datagen.fit(X_train)

# Callbacks for fine-tuning
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)

# Final check before training
assert X_train.shape[0] == y_age_train.shape[0] == y_gender_train.shape[0] == y_ethnicity_train.shape[0], "Mismatch in training data sizes!"
assert X_test.shape[0] == y_age_test.shape[0] == y_gender_test.shape[0] == y_ethnicity_test.shape[0], "Mismatch in testing data sizes!"

# Continue training (fine-tuning)
model.fit(
    datagen.flow(X_train, {"age_output": y_age_train, "gender_output": y_gender_train, "ethnicity_output": y_ethnicity_train}, batch_size=64),
    validation_data=(X_test, {"age_output": y_age_test, "gender_output": y_gender_test, "ethnicity_output": y_ethnicity_test}),
    epochs=30,  # Increase epochs for fine-tuning
    steps_per_epoch=len(X_train) // 64,  # Ensures full dataset coverage
    class_weight={"gender_output": gender_class_weights},
    callbacks=[lr_scheduler, early_stopping]
)

# Save the fine-tuned model
model.save("multi_output_cnn_finetuned.keras")

print("Fine-tuning complete. Model saved as 'multi_output_cnn_finetuned.keras'")


X shape: (23705, 48, 48, 1)
y_age shape: (23705,)
y_gender shape: (23705,)
y_ethnicity shape: (23705, 5)


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (18964, 48, 48, 1), y.shape = ()

In [11]:
model.load("multi_output_cnn.keras")

AttributeError: 'Functional' object has no attribute 'load'

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D,
                                     Dense, Dropout, BatchNormalization, RandomFlip,
                                     RandomRotation, RandomZoom, RandomContrast)
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Load and preprocess data
def load_data():
    df = pd.read_csv("age_gender.csv")
    
    # Process pixels
    df["pixels"] = df["pixels"].apply(
        lambda x: np.array(x.split(), dtype="uint8").reshape(48, 48, 1)
    )
    
    # Normalize pixel values
    X = np.stack(df["pixels"].values) / 255.0
    
    # Process labels
    max_age = df["age"].max()
    y_age = df["age"].values / max_age  # Normalize age 0-1
    y_gender = df["gender"].values
    y_ethnicity = to_categorical(df["ethnicity"].values)
    
    return X, y_age, y_gender, y_ethnicity, max_age

# Build enhanced model
def build_multi_task_model(input_shape=(48, 48, 1), num_ethnicities=5):
    # Data augmentation
    data_augmentation = tf.keras.Sequential([
        RandomFlip("horizontal"),
        RandomRotation(0.1),
        RandomZoom(0.2),
        RandomContrast(0.1),
    ])
    
    # Input and augmentation
    input_layer = Input(shape=input_shape)
    x = data_augmentation(input_layer)
    
    # Feature extractor
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = MaxPooling2D(2, 2)(x)
    
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = MaxPooling2D(2, 2)(x)
    
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = GlobalAveragePooling2D()(x)
    
    # Shared dense layers
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = Dropout(0.5)(x)
    
    # Task-specific branches
    # Age branch
    age_branch = Dense(256)(x)
    age_branch = BatchNormalization()(age_branch)
    age_branch = tf.keras.layers.LeakyReLU(0.2)(age_branch)
    age_branch = Dropout(0.3)(age_branch)
    age_output = Dense(1, activation='linear', name='age_output')(age_branch)
    
    # Gender branch
    gender_branch = Dense(128)(x)
    gender_branch = BatchNormalization()(gender_branch)
    gender_branch = tf.keras.layers.LeakyReLU(0.2)(gender_branch)
    gender_branch = Dropout(0.3)(gender_branch)
    gender_output = Dense(1, activation='sigmoid', name='gender_output')(gender_branch)
    
    # Ethnicity branch
    ethnicity_branch = Dense(256)(x)
    ethnicity_branch = BatchNormalization()(ethnicity_branch)
    ethnicity_branch = tf.keras.layers.LeakyReLU(0.2)(ethnicity_branch)
    ethnicity_branch = Dropout(0.4)(ethnicity_branch)
    ethnicity_output = Dense(num_ethnicities, activation='softmax', name='ethnicity_output')(ethnicity_branch)
    
    return Model(inputs=input_layer, outputs=[age_output, gender_output, ethnicity_output])

# Main training function
def train_model():
    # Load and prepare data
    X, y_age, y_gender, y_ethnicity, max_age = load_data()
    
    # Split data
    (X_train, X_test,
     y_age_train, y_age_test,
     y_gender_train, y_gender_test,
     y_ethnicity_train, y_ethnicity_test) = train_test_split(
        X, y_age, y_gender, y_ethnicity,
        test_size=0.2, random_state=42
    )
    
    # Build model
    model = build_multi_task_model()
    
    # Calculate class weights
    ethnicity_classes = np.argmax(y_ethnicity_train, axis=1)
    ethnicity_weights = compute_class_weight('balanced', classes=np.unique(ethnicity_classes), y=ethnicity_classes)
    ethnicity_weights = dict(enumerate(ethnicity_weights))
    
    gender_weights = compute_class_weight('balanced', classes=[0, 1], y=y_gender_train)
    gender_weights = dict(enumerate(gender_weights))
    
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss={
            'age_output': 'mae',
            'gender_output': 'binary_crossentropy',
            'ethnicity_output': 'categorical_crossentropy'
        },
        loss_weights={
            'age_output': 0.3,
            'gender_output': 0.7,
            'ethnicity_output': 1.0
        },
        metrics={
            'age_output': ['mae', 'mse'],
            'gender_output': ['accuracy'],
            'ethnicity_output': ['accuracy']
        }
    )
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=8,
            min_lr=1e-7
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.keras',
            save_best_only=True,
            monitor='val_loss'
        )
    ]
    
    # Train
    history = model.fit(
        X_train,
        {
            'age_output': y_age_train,
            'gender_output': y_gender_train,
            'ethnicity_output': y_ethnicity_train
        },
        validation_data=(
            X_test,
            {
                'age_output': y_age_test,
                'gender_output': y_gender_test,
                'ethnicity_output': y_ethnicity_test
            }
        ),
        epochs=30,
        batch_size=64,
        callbacks=callbacks,
        class_weight={
            'gender_output': gender_weights,
            'ethnicity_output': ethnicity_weights
        }
    )
    
    # Save final model
    model.save("multi_task_model.keras")
    print("Model training complete and saved as multi_task_model.keras")
    
    return model, max_age

# Load trained model and make predictions
def load_and_predict(model_path, max_age):
    model = load_model(model_path)
    
    def predict(image):
        """Input: normalized image array (48x48x1)"""
        predictions = model.predict(np.expand_dims(image, axis=0))
        age = predictions[0][0][0] * max_age
        gender = "Male" if predictions[1][0][0] > 0.5 else "Female"
        ethnicity = np.argmax(predictions[2][0])
        return age, gender, ethnicity
    
    return predict

# Run training
if __name__ == "__main__":
    trained_model, max_age = train_model()
    
    # Example usage after training
    predictor = load_and_predict("multi_task_model.keras", max_age)
    
    # Load a sample image (replace with your image)
    sample_image = X_test[0]  # From test set
    age, gender, ethnicity = predictor(sample_image)
    print(f"Predicted Age: {age:.1f} years")
    print(f"Predicted Gender: {gender}")
    print(f"Predicted Ethnicity: {ethnicity}")

InvalidParameterError: The 'classes' parameter of compute_class_weight must be an instance of 'numpy.ndarray'. Got [0, 1] instead.

In [19]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("multi_output_cnn.keras")

C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [23]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_image(image_path):
    img = load_img(image_path, color_mode="grayscale", target_size=(48, 48))
    img = img_to_array(img) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Path to your test image
image_path = "s.jpg"
input_image = preprocess_image(image_path)

# Make prediction
age_pred, gender_pred, ethnicity_pred = model.predict(input_image)

# Process results
age = round(age_pred[0][0])
gender = "Female" if gender_pred[0][0] > 0.5 else "Male"
ethnicity = np.argmax(ethnicity_pred[0])

print(f"Predicted Age: {age}")
print(f"Predicted Gender: {gender}")
print(f"Predicted Ethnicity Class: {ethnicity}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicted Age: 35
Predicted Gender: Female
Predicted Ethnicity Class: 0


In [27]:
import os

# Create directory if it doesn't exist
os.makedirs("saved_models", exist_ok=True)

# Modified ModelCheckpoint callback
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=8,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'saved_models/multi_output_cnn.keras',  # Path updated
        save_best_only=True,
        monitor='val_loss'
    )
]

# Modified model saving
model.save("saved_models/multi_output_cnn.keras")

In [29]:
# At the top of your file
import os

# Create model directory if not exists
MODEL_DIR = "saved_models"
os.makedirs(MODEL_DIR, exist_ok=True)

# In your training function
callbacks = [
    # ... other callbacks ...
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(MODEL_DIR, "best_model.keras"),
        save_best_only=True,
        monitor='val_loss'
    )
]

# After training
final_model_path = os.path.join(MODEL_DIR, "multi_output_cnn.keras")
model.save(final_model_path)

In [31]:
def load_and_predict(model_path, max_age):
    model = load_model(model_path)
    # ... rest of the function ...
    
# Usage example
predictor = load_and_predict("saved_models/multi_output_cnn.keras", max_age)

NameError: name 'max_age' is not defined

In [33]:
import os  # Add this import at the top

# ... [keep all your existing code until the end] ...

# Create a directory for saving models if it doesn't exist
model_dir = "saved_models"
os.makedirs(model_dir, exist_ok=True)

# Save the model in the directory
model.save(os.path.join(model_dir, "multi_output_cnn.keras"))

In [ ]:
# Add this with your other imports
from tensorflow.keras.callbacks import ModelCheckpoint

# Create checkpoint callback
checkpoint_path = os.path.join(model_dir, "checkpoints", "model-{epoch:03d}.keras")
os.makedirs(os.path.join(model_dir, "checkpoints"), exist_ok=True)

checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    save_best_only=False,
    verbose=1
)

# Modify your fit call to include the callback
model.fit(
    X_train,
    {"age_output": y_age_train, "gender_output": y_gender_train, "ethnicity_output": y_ethnicity_train},
    validation_data=(X_test, {"age_output": y_age_test, "gender_output": y_gender_test, "ethnicity_output": y_ethnicity_test}),
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint]  # Add this
)

Epoch 1/20
592/593 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - age_output_loss: 32.3924 - age_output_mae: 2.5274 - ethnicity_output_accuracy: 0.4034 - ethnicity_output_loss: 1.6965 - gender_output_accuracy: 0.5806 - gender_output_loss: 0.7076 - loss: 34.7965
Epoch 1: saving model to saved_models\checkpoints\model-001.keras
593/593 ━━━━━━━━━━━━━━━━━━━━ 50s 78ms/step - age_output_loss: 32.3068 - age_output_mae: 2.5233 - ethnicity_output_accuracy: 0.4035 - ethnicity_output_loss: 1.6959 - gender_output_accuracy: 0.5808 - gender_output_loss: 0.7074 - loss: 34.7101 - val_age_output_loss: 0.3886 - val_age_output_mae: 0.5738 - val_ethnicity_output_accuracy: 0.5275 - val_ethnicity_output_loss: 1.1493 - val_gender_output_accuracy: 0.7941 - val_gender_output_loss: 0.5077 - val_loss: 2.0480
Epoch 2/20
592/593 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - age_output_loss: 0.7653 - age_output_mae: 0.7055 - ethnicity_output_accuracy: 0.5248 - ethnicity_output_loss: 1.2125 - gender_output_accuracy: 0.7369 - gender_outp